## ⚠️ 重要な更新情報

**🚀 新しい改善版ランチャーが利用可能です！**

Google Colab環境での**依存関係インストールのハングアップ問題**を根本的に解決するため、新しい`launcher_colab.ipynb`を作成しました。

### 📋 主要な改善点
- ✅ **段階的依存関係インストール**: `pip install -r requirements.txt`のハングアップを解決
- ✅ **環境構築と実験実行の分離**: 問題の切り分けが容易
- ✅ **Git sparse-checkout**: 必要ファイルのみ高速取得
- ✅ **包括的デバッグ支援**: エラー時の迅速な解決

### 🎯 推奨使用方法
1. **新規実験**: `launcher_colab.ipynb`を使用してください
2. **既存実験**: このノートブックも引き続き使用可能ですが、問題発生時は新版を推奨

### 📁 ファイル構成
```
launcher_colab.ipynb          # 🆕 推奨: メインランチャー
requirements-colab.txt        # 🆕 Colab最適化版依存関係
sycophancy_analysis_colab.ipynb  # このファイル（引き続き使用可能）
README_COLAB_IMPROVEMENTS.md # 🆕 詳細な改善ガイド
```

---

# 🧠 LLM迎合性分析 - Google Colab版

このノートブックは、Google Colab環境で**Git sparse checkout**を使用してLLMの迎合性（Sycophancy）分析を実行します。

## 🎯 特徴
- **軽量設計**: メインコードをGitリポジトリから直接インポート
- **自動更新**: 最新のコード改善が自動反映
- **Gemma-2B対応**: 効率的な2Bパラメータモデルをサポート
- **T4 GPU最適化**: Google Colab T4環境での最適パフォーマンス

## 📋 実行順序
1. **環境セットアップ**: Git clone & 依存関係インストール
2. **モデル選択**: 使用するLLMモデルを選択
3. **分析実行**: 迎合性分析の実行
4. **結果可視化**: 分析結果の詳細表示

---

**⚠️ 重要**: このノートブックはGoogle Colab環境での使用を前提としています。

## 🎯 使用方法とカスタマイズ

### ✅ **基本的な使用手順**

1. **環境セットアップセル**を実行
   - Git sparse checkoutでコードを取得
   - 必要な依存関係をインストール

2. **メインコードインポートセル**でモデルを選択
   ```python
   # 使用したいモデル設定を選択
   SELECTED_CONFIG = GEMMA2B_TEST_CONFIG     # Gemma-2B
   # SELECTED_CONFIG = LLAMA3_TEST_CONFIG    # Llama-3.2-1B  
   ```

3. **分析実行セル**で迎合性分析を実行
   - 全て自動で実行（モデル読み込み〜分析〜結果保存）

4. **可視化セル**で結果を確認
   - インタラクティブなグラフ表示
   - 詳細統計と具体例の表示

---

### 🔧 **高度なカスタマイズ**

#### **モデル変更**
```python
# Gemma-2B（推奨 - T4 GPUに最適）
SELECTED_CONFIG = GEMMA2B_TEST_CONFIG          # テスト用（少数サンプル）
SELECTED_CONFIG = GEMMA2B_MEMORY_OPTIMIZED_CONFIG  # メモリ効率重視

# Llama-3.2-1B
SELECTED_CONFIG = LLAMA3_TEST_CONFIG           # テスト用
SELECTED_CONFIG = LLAMA3_MEMORY_OPTIMIZED_CONFIG   # メモリ効率重視

# 自動選択（環境に応じて最適な設定）
SELECTED_CONFIG = get_auto_config()
```

#### **サンプルサイズ調整**
```python
# config.pyから設定をコピーして調整
from copy import deepcopy
custom_config = deepcopy(GEMMA2B_TEST_CONFIG)
custom_config.data.sample_size = 100  # サンプル数を変更
SELECTED_CONFIG = custom_config
```

#### **詳細デバッグ**
```python
# デバッグモードを有効化
custom_config.debug.verbose = True
custom_config.debug.show_prompts = True
custom_config.debug.show_responses = True
```

---

### 📊 **対応モデル**

| モデル | 推奨設定 | メモリ使用量 | 特徴 |
|--------|----------|--------------|------|
| **Gemma-2B** | `GEMMA2B_TEST_CONFIG` | ~4-6GB | T4最適、高効率 |
| **Llama-3.2-1B** | `LLAMA3_TEST_CONFIG` | ~3-5GB | 軽量、高性能 |  
| **Llama-3.2-3B** | `LLAMA3_MEMORY_OPTIMIZED_CONFIG` | ~8-12GB | 高性能、要量子化 |

---

### ⚠️ **トラブルシューティング**

**Git cloneエラー:**
- ネットワーク接続を確認
- リポジトリのアクセス権限を確認

**メモリ不足エラー:**
- より小さなモデル（Gemma-2B）を使用
- メモリ最適化設定を選択
- サンプルサイズを削減

**インポートエラー:**
- 環境セットアップセルを再実行
- ランタイムを再起動

---

### 🔄 **更新とメンテナンス**

このノートブックは**Git sparse checkout**を使用しているため：

✅ **自動更新**: メインリポジトリの改善が自動反映  
✅ **軽量**: 必要ファイルのみダウンロード  
✅ **最新**: 常に最新のバグ修正と機能改善を利用  

メインコードに更新がある場合は、環境セットアップセルを再実行するだけで最新版が取得されます。

In [ ]:
# 🔧 環境セットアップ & Git Sparse Checkout
import os
import sys
import subprocess
import torch
import warnings

def setup_colab_environment():
    """Google Colab環境のセットアップとGit sparse checkoutの実行"""
    
    # Google Colab環境の確認
    try:
        import google.colab
        print("✅ Google Colab環境を検出")
        IN_COLAB = True
    except ImportError:
        print("⚠️  ローカル環境での実行を検出")
        IN_COLAB = False
    
    # GPU情報の表示
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"🚀 GPU検出: {gpu_name} ({gpu_memory:.1f}GB)")
    else:
        print("❌ GPU利用不可 - CPUモードで実行")
    
    return IN_COLAB

def git_sparse_checkout(branch="main"):
    """Git sparse checkoutでリポジトリから必要ファイルのみを取得（改良版）"""
    
    print(f"📂 Git sparse checkoutを開始... (ブランチ: {branch})")
    
    # 現在のディレクトリを保存（Google Colab対応）
    original_dir = os.getcwd()
    print(f"🔍 元のディレクトリ: {original_dir}")
    
    # 既存のクローンディレクトリがあれば削除（正しいディレクトリ名を使用）
    if os.path.exists("my_sae"):
        !rm -rf my_sae
        print("🧹 既存のクローンディレクトリを削除")
    
    try:
        # リポジトリのクローン（filter=blob:noneで高速化、checkoutなし）
        print(f"📥 リポジトリをクローン中... (ブランチ: {branch})")
        !git clone --filter=blob:none --no-checkout -b {branch} https://github.com/Itsuki966/my_sae.git
        
        # ディレクトリ移動
        os.chdir("my_sae")
        print(f"🔍 作業ディレクトリ変更: {os.getcwd()}")

        # Sparse checkout の初期化（coneモードで高速化）
        print("🔧 Sparse checkout を設定中...")
        !git sparse-checkout init --cone
        
        # 必要なファイル・ディレクトリを指定（改良版requirements-colab.txtを含む）
        files_to_checkout = [
            "sycophancy_analyzer.py",
            "config.py", 
            "memory_optimizer.py",
            "eval_dataset",
            "requirements-colab.txt",  # 最適化版を優先使用
            "requirements.txt",        # フォールバック用
            "pyproject.toml"           # 参考情報用
        ]
        
        checkout_list = " ".join(files_to_checkout)
        !git sparse-checkout set {checkout_list}
        
        # 実際のチェックアウト実行
        print("📋 必要ファイルをチェックアウト中...")
        !git checkout
        
        # 現在のコミットハッシュを表示（実験再現性のため）
        result = !git rev-parse --short HEAD
        commit_hash = result[0] if result else "unknown"
        print(f"📌 使用コミット: {commit_hash}")
        
        # 元のディレクトリに戻る（重要：Google Colab対応）
        os.chdir(original_dir)
        print(f"🔍 ディレクトリ復元: {os.getcwd()}")
        
        # チェックアウトされたファイルの確認
        print(f"\n✅ 取得完了ファイル:")
        for item in files_to_checkout:
            item_path = os.path.join("my_sae", item)
            if os.path.exists(item_path):
                if os.path.isfile(item_path):
                    size = os.path.getsize(item_path) / 1024  # KB
                    print(f"   📄 {item} ({size:.1f}KB)")
                else:
                    try:
                        file_count = len([f for f in os.listdir(item_path) if os.path.isfile(os.path.join(item_path, f))])
                        print(f"   📁 {item}/ ({file_count}ファイル)")
                    except:
                        print(f"   📁 {item}/ (ディレクトリ)")
            else:
                print(f"   ❌ {item} (見つかりません)")
        
        return True, commit_hash
        
    except Exception as e:
        print(f"❌ Git sparse checkout エラー: {e}")
        print(f"💡 ブランチ'{branch}'が存在するか確認してください")
        print(f"💡 ネットワーク接続を確認してください")
        # エラー時も元のディレクトリに戻る
        try:
            os.chdir(original_dir)
        except:
            pass
        return False, None

def install_dependencies():
    """必要な依存関係の段階的インストール（ハングアップ問題対策版）"""
    
    print("📦 依存関係をインストール中...")
    
    # Colab最適化版requirements.txtを優先使用
    colab_requirements = "my_sae/requirements-colab.txt"
    fallback_requirements = "my_sae/requirements.txt"
    
    # 段階的インストール戦略
    if os.path.exists(colab_requirements):
        print(f"🎯 Colab最適化版を使用: {colab_requirements}")
        strategy = "colab_optimized"
    elif os.path.exists(fallback_requirements):
        print(f"📋 標準版を使用: {fallback_requirements}")
        strategy = "standard"
    else:
        print("⚠️ requirements.txtが見つかりません - 手動インストールに切り替え")
        strategy = "manual"
    
    success = False
    
    if strategy == "colab_optimized":
        success = install_colab_optimized(colab_requirements)
    elif strategy == "standard":
        success = install_with_timeout_protection(fallback_requirements)
    else:  # manual
        success = install_core_manually()
    
    if success:
        print("✅ 依存関係インストール完了")
        return True
    else:
        print("❌ 依存関係インストールに失敗")
        print("💡 手動インストール方法:")
        print("   1. 上記のエラーメッセージを確認")
        print("   2. 個別にライブラリをインストール: !pip install <package_name>")
        print("   3. 必要に応じてランタイムを再起動")
        return False

def install_colab_optimized(requirements_file):
    """Colab最適化版requirements.txtを使用したインストール"""
    print("🚀 Colab最適化版依存関係をインストール中...")
    
    try:
        # タイムアウト保護付きでインストール
        import subprocess
        import time
        
        start_time = time.time()
        
        # 最大20分のタイムアウト
        result = subprocess.run(
            ['pip', 'install', '-r', requirements_file],
            timeout=1200,  # 20分
            capture_output=True,
            text=True
        )
        
        elapsed = time.time() - start_time
        
        if result.returncode == 0:
            print(f"✅ Colab最適化版インストール完了 ({elapsed:.1f}s)")
            return True
        else:
            print(f"❌ Colab最適化版インストール失敗")
            if result.stderr:
                print(f"エラー: {result.stderr[:300]}...")
            return False
            
    except subprocess.TimeoutExpired:
        print("⏰ Colab最適化版インストール - タイムアウト (20分)")
        print("💡 手動インストールに切り替えます...")
        return install_core_manually()
    except Exception as e:
        print(f"❌ Colab最適化版インストール例外: {e}")
        return False

def install_with_timeout_protection(requirements_file):
    """タイムアウト保護付きの標準インストール"""
    print("⚠️ 標準版requirements.txtを使用（タイムアウト保護付き）...")
    
    try:
        import subprocess
        import time
        
        start_time = time.time()
        
        # 最大30分のタイムアウト
        result = subprocess.run(
            ['pip', 'install', '-r', requirements_file],
            timeout=1800,  # 30分
            capture_output=True,
            text=True
        )
        
        elapsed = time.time() - start_time
        
        if result.returncode == 0:
            print(f"✅ 標準版インストール完了 ({elapsed:.1f}s)")
            return True
        else:
            print(f"❌ 標準版インストール失敗")
            if result.stderr:
                print(f"エラー: {result.stderr[:300]}...")
            print("💡 手動インストールに切り替えます...")
            return install_core_manually()
            
    except subprocess.TimeoutExpired:
        print("⏰ 標準版インストール - タイムアウト (30分)")
        print("💡 手動インストールに切り替えます...")
        return install_core_manually()
    except Exception as e:
        print(f"❌ 標準版インストール例外: {e}")
        return install_core_manually()

def install_core_manually():
    """核心ライブラリの手動段階的インストール"""
    print("🔧 核心ライブラリを手動でインストール中...")
    
    # 必須ライブラリのリスト（優先順位順）
    core_libraries = [
        ("torch", "PyTorch (GPU対応)"),
        ("transformers", "HuggingFace Transformers"),
        ("sae-lens", "SAE Lens (核心ライブラリ)"),
        ("accelerate", "メモリ効率化"),
        ("plotly", "可視化"),
        ("tqdm", "プログレスバー")
    ]
    
    failed_libraries = []
    
    for library, description in core_libraries:
        print(f"📦 {description} をインストール中...")
        try:
            !pip install -q {library}
            print(f"   ✅ {library} 完了")
        except Exception as e:
            print(f"   ❌ {library} 失敗: {e}")
            failed_libraries.append(library)
    
    if failed_libraries:
        print(f"⚠️ 失敗したライブラリ: {', '.join(failed_libraries)}")
        print("💡 これらのライブラリは後で個別に再試行してください")
        return len(failed_libraries) < len(core_libraries) // 2  # 半数以上成功すれば継続
    else:
        print("✅ 全ての核心ライブラリのインストール完了")
        return True

# 🔄 実験設定（ブランチ選択）
# ===============================================
EXPERIMENT_BRANCH = "main"  # ここを変更して異なるブランチを使用
# EXPERIMENT_BRANCH = "feature/new-model-support"    # 新機能ブランチ
# EXPERIMENT_BRANCH = "experiment/temperature-test"  # 実験ブランチ
# EXPERIMENT_BRANCH = "develop"                       # 開発ブランチ

# 環境セットアップの実行
print("🚀 Google Colab環境セットアップを開始...")
print("=" * 60)

IN_COLAB = setup_colab_environment()

# Git sparse checkoutの実行
success, commit_hash = git_sparse_checkout(EXPERIMENT_BRANCH)

if success:
    print(f"\n📦 依存関係をインストール中...")
    install_dependencies()
    
    # パスの追加（インポート用）- Google Colab環境対応
    repo_path = os.path.abspath("my_sae")
    if repo_path not in sys.path:
        sys.path.insert(0, repo_path)
        print(f"📁 Pythonパスに追加: {repo_path}")
    
    # Colab環境では /content/ もパスに追加
    if IN_COLAB:
        content_path = "/content"
        content_repo_path = "/content/my_sae"
        if content_path not in sys.path:
            sys.path.insert(0, content_path)
        if content_repo_path not in sys.path:
            sys.path.insert(0, content_repo_path)
        print(f"📁 Colab用パス追加: {content_path}, {content_repo_path}")
    
    print(f"\n" + "=" * 60)
    print("✅ 環境セットアップ完了！")
    print(f"🔖 使用ブランチ: {EXPERIMENT_BRANCH}")
    print(f"📌 コミットハッシュ: {commit_hash}")
    print("💡 次のセルでメインコードをインポートしてください")
    
    # 実験記録用のグローバル変数
    EXPERIMENT_METADATA = {
        'branch': EXPERIMENT_BRANCH,
        'commit_hash': commit_hash,
        'setup_time': __import__('datetime').datetime.now().isoformat()
    }
    
else:
    print(f"\n❌ セットアップに失敗しました")
    print(f"💡 ネットワーク接続またはブランチ'{EXPERIMENT_BRANCH}'の存在を確認してください")

In [ ]:
# 📚 メインコードのインポートと設定
try:
    # メインの分析クラスをインポート
    from sycophancy_analyzer import SycophancyAnalyzer
    
    # 設定をインポート
    from config import (
        GEMMA2B_TEST_CONFIG, GEMMA2B_PROD_CONFIG, GEMMA2B_MEMORY_OPTIMIZED_CONFIG,
        LLAMA3_TEST_CONFIG, LLAMA3_MEMORY_OPTIMIZED_CONFIG,
        TEST_CONFIG, DEFAULT_CONFIG,
        get_auto_config
    )
    
    print("✅ メインコードのインポート完了")
    print("📋 利用可能な設定:")
    print("   🔸 GEMMA2B_TEST_CONFIG - Gemma-2B テスト用（少数サンプル）")
    print("   🔸 GEMMA2B_PROD_CONFIG - Gemma-2B 本番用（大規模分析）") 
    print("   🔸 GEMMA2B_MEMORY_OPTIMIZED_CONFIG - Gemma-2B メモリ最適化版")
    print("   🔸 LLAMA3_TEST_CONFIG - Llama-3.2-1B テスト用")
    print("   🔸 LLAMA3_MEMORY_OPTIMIZED_CONFIG - Llama-3.2-1B メモリ最適化版")
    print("   🔸 TEST_CONFIG - GPT-2 テスト用")
    print("   🔸 get_auto_config() - 環境に応じた自動選択")
    
except ImportError as e:
    print(f"❌ インポートエラー: {e}")
    print("💡 上記の環境セットアップセルを先に実行してください")
    raise

# 🎯 高速実験設定（ここを変更して即座に実験条件を変更）
# ===============================================

# 🔧 実験設定のカスタマイズ機能
def create_custom_config(base_config, **kwargs):
    """設定を動的にカスタマイズする関数"""
    from copy import deepcopy
    
    custom_config = deepcopy(base_config)
    
    # 動的に設定を変更
    if 'sample_size' in kwargs:
        custom_config.data.sample_size = kwargs['sample_size']
    if 'model_name' in kwargs:
        custom_config.model.name = kwargs['model_name']
    if 'max_new_tokens' in kwargs:
        custom_config.generation.max_new_tokens = kwargs['max_new_tokens']
    if 'temperature' in kwargs:
        custom_config.generation.temperature = kwargs['temperature']
    if 'verbose' in kwargs:
        custom_config.debug.verbose = kwargs['verbose']
    
    return custom_config

# 🚀 実験設定（この部分を変更して即座に実験条件を変更）
# ===============================================

# 基本設定の選択
BASE_CONFIG = GEMMA2B_TEST_CONFIG  # ベース設定

# 実験パラメータの上書き（ここを自由に変更）
EXPERIMENT_PARAMS = {
    'sample_size': 10,        # サンプル数（1-1000）
    'temperature': 0.3,       # 生成温度（0.0-1.0）
    'max_new_tokens': 3,      # 最大生成トークン数
    'verbose': True,          # 詳細ログ出力
}

# カスタム設定の作成
SELECTED_CONFIG = create_custom_config(BASE_CONFIG, **EXPERIMENT_PARAMS)

print(f"\n🎯 実験設定:")
print(f"   📱 ベースモデル: {BASE_CONFIG.model.name}")
print(f"   ? サンプル数: {SELECTED_CONFIG.data.sample_size}")
print(f"   🌡️  温度: {SELECTED_CONFIG.generation.temperature}")
print(f"   ? 最大トークン: {SELECTED_CONFIG.generation.max_new_tokens}")
print(f"   🔍 詳細ログ: {SELECTED_CONFIG.debug.verbose}")
print(f"   🔍 SAE: {SELECTED_CONFIG.model.sae_release}")
print(f"   🖥️  デバイス: {SELECTED_CONFIG.model.device}")

# 🔄 ワンクリック実験変更例
print(f"\n🔄 高速実験変更例:")
print(f"# 大規模実験に変更")
print(f"EXPERIMENT_PARAMS['sample_size'] = 100")
print(f"")
print(f"# 高温度生成実験")  
print(f"EXPERIMENT_PARAMS['temperature'] = 0.8")
print(f"")
print(f"# より長い生成実験")
print(f"EXPERIMENT_PARAMS['max_new_tokens'] = 10")

# メモリ最適化設定
warnings.filterwarnings('ignore')
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

print(f"\n✅ 設定完了 - 次のセルで分析を実行してください")
print(f"💡 EXPERIMENT_PARAMSを変更してこのセルを再実行すると即座に設定が変更されます")

In [ ]:
# 🔬 LLM迎合性分析の実行
print("🚀 LLM迎合性分析を開始...")
print("=" * 60)

# 実験メタデータの準備
experiment_metadata = {
    'experiment_start': __import__('datetime').datetime.now().isoformat(),
    'branch': EXPERIMENT_METADATA.get('branch', 'unknown') if 'EXPERIMENT_METADATA' in globals() else 'unknown',
    'commit_hash': EXPERIMENT_METADATA.get('commit_hash', 'unknown') if 'EXPERIMENT_METADATA' in globals() else 'unknown',
    'config_params': {
        'model_name': SELECTED_CONFIG.model.name,
        'sample_size': SELECTED_CONFIG.data.sample_size,
        'temperature': SELECTED_CONFIG.generation.temperature,
        'max_new_tokens': SELECTED_CONFIG.generation.max_new_tokens,
    }
}

print(f"📋 実験メタデータ:")
print(f"   🌿 ブランチ: {experiment_metadata['branch']}")
print(f"   📌 コミット: {experiment_metadata['commit_hash']}")
print(f"   ⏰ 開始時刻: {experiment_metadata['experiment_start']}")
print(f"   📊 設定: {experiment_metadata['config_params']}")

try:
    # 分析器の初期化
    print(f"\n🔧 分析器を初期化中...")
    analyzer = SycophancyAnalyzer(SELECTED_CONFIG)
    
    # メモリ使用状況の表示
    if torch.cuda.is_available():
        memory_used = torch.cuda.memory_allocated(0) / 1024**3
        memory_total = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"💾 初期GPU Memory: {memory_used:.1f}GB / {memory_total:.1f}GB")
        experiment_metadata['initial_memory_gb'] = round(memory_used, 2)
    
    # 完全分析の実行（モデル読み込み + 分析 + 結果保存を一括実行）
    print(f"\n🔄 完全分析を実行中...")
    print(f"   📋 この処理には数分かかる場合があります...")
    
    # 実行時間測定開始
    import time
    start_time = time.time()
    
    # メイン分析の実行
    analyzer.run_complete_analysis()
    
    # 実行時間測定終了
    end_time = time.time()
    execution_time = end_time - start_time
    experiment_metadata['execution_time_seconds'] = round(execution_time, 2)
    experiment_metadata['experiment_end'] = __import__('datetime').datetime.now().isoformat()
    
    print(f"\n" + "=" * 60)
    print("✅ 分析完了！")
    print(f"⏱️  実行時間: {execution_time:.1f}秒")
    
    # 結果の簡易表示
    if hasattr(analyzer, 'analysis_results') and analyzer.analysis_results:
        stats = analyzer.analysis_results
        
        # 実験メタデータに結果を追加
        experiment_metadata['results'] = {
            'total_samples': stats.get('total_samples', 0),
            'sycophancy_rate': stats.get('sycophancy_rate', 0),
            'initial_accuracy': stats.get('initial_accuracy', 0),
            'challenge_accuracy': stats.get('challenge_accuracy', 0),
        }
        
        print(f"\n📊 分析結果サマリー:")
        print(f"   🎯 使用モデル: {stats.get('model_name', 'N/A')}")
        print(f"   📝 総サンプル数: {stats.get('total_samples', 0)}")
        print(f"   📈 迎合率: {stats.get('sycophancy_rate', 0):.1%}")
        print(f"   📈 初回正答率: {stats.get('initial_accuracy', 0):.1%}")
        print(f"   📈 チャレンジ後正答率: {stats.get('challenge_accuracy', 0):.1%}")
    
    # メモリ使用状況の最終確認
    if torch.cuda.is_available():
        memory_used = torch.cuda.memory_allocated(0) / 1024**3
        print(f"\n💾 最終GPU Memory: {memory_used:.1f}GB / {memory_total:.1f}GB")
        experiment_metadata['final_memory_gb'] = round(memory_used, 2)
    
    # 実験ログの保存
    import json
    experiment_log_file = f"experiment_log_{experiment_metadata['experiment_start'][:19].replace(':', '-')}.json"
    with open(experiment_log_file, 'w', encoding='utf-8') as f:
        json.dump(experiment_metadata, f, indent=2, ensure_ascii=False)
    
    print(f"\n📋 実験ログ保存: {experiment_log_file}")
    print(f"💡 詳細な結果は次のセルで可視化されます")
    
    # グローバル変数に結果を保存（可視化用）
    analysis_results = analyzer.analysis_results if hasattr(analyzer, 'analysis_results') else {}
    detailed_results = analyzer.results if hasattr(analyzer, 'results') else []
    
    # 実験メタデータもグローバルに保存
    current_experiment_metadata = experiment_metadata
    
except Exception as e:
    print(f"❌ 分析実行エラー: {e}")
    print(f"\n💡 トラブルシューティング:")
    print(f"   1. GPU メモリが不足していないか確認")
    print(f"   2. より小さなサンプルサイズの設定を試行")
    print(f"   3. ランタイムを再起動して再実行")
    print(f"   4. 異なるブランチを試行（EXPERIMENT_BRANCH変更）")
    
    # エラー情報を実験メタデータに記録
    experiment_metadata['error'] = str(e)
    experiment_metadata['status'] = 'failed'
    
    # エラー時のメモリクリア
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    raise

print(f"\n🎉 分析セクション完了 - 次のセルで結果を可視化してください")

In [ ]:
# 📊 分析結果の詳細可視化
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

# 結果データの確認
try:
    if 'analysis_results' in globals() and 'detailed_results' in globals():
        stats = analysis_results
        results = detailed_results
        
        print("📈 分析結果の可視化を開始...")
        print("=" * 50)
        
        if not stats or not results:
            print("❌ 可視化用データが不足しています")
            print("💡 上記の分析実行セルを先に実行してください")
        else:
            # 1. 基本統計の表示
            print(f"📊 詳細統計:")
            print(f"   🎯 使用モデル: {stats.get('model_name', 'N/A')}")
            print(f"   📝 総サンプル数: {stats.get('total_samples', 0)}")
            print(f"   🔄 迎合事例数: {stats.get('sycophantic_responses', 0)}")
            print(f"   📈 迎合率: {stats.get('sycophancy_rate', 0):.1%}")
            print(f"   📈 初回正答率: {stats.get('initial_accuracy', 0):.1%}")
            print(f"   📈 チャレンジ後正答率: {stats.get('challenge_accuracy', 0):.1%}")
            
            # 2. インタラクティブな可視化（Plotly）
            fig = make_subplots(
                rows=2, cols=2,
                subplot_titles=['迎合率分析', '正答率比較', '回答変化パターン', '迎合時の正答率'],
                specs=[[{"type": "pie"}, {"type": "bar"}],
                       [{"type": "bar"}, {"type": "bar"}]]
            )
            
            # サブプロット1: 迎合率（パイチャート）
            sycophantic_count = stats.get('sycophantic_responses', 0)
            consistent_count = stats.get('total_samples', 0) - sycophantic_count
            
            fig.add_trace(go.Pie(
                labels=['一貫した回答', '迎合的回答'],
                values=[consistent_count, sycophantic_count],
                hole=0.3
            ), row=1, col=1)
            
            # サブプロット2: 正答率比較
            accuracies = [stats.get('initial_accuracy', 0), stats.get('challenge_accuracy', 0)]
            fig.add_trace(go.Bar(
                x=['初回回答', 'チャレンジ後'],
                y=accuracies,
                text=[f'{acc:.1%}' for acc in accuracies],
                textposition='auto'
            ), row=1, col=2)
            
            # サブプロット3: 回答変化パターン
            if results:
                pattern_counts = {'正解→正解': 0, '正解→不正解': 0, '不正解→正解': 0, '不正解→不正解': 0}
                for result in results:
                    initial_correct = result.get('is_initial_correct', False)
                    challenge_correct = result.get('is_challenge_correct', False)
                    
                    if initial_correct and challenge_correct:
                        pattern_counts['正解→正解'] += 1
                    elif initial_correct and not challenge_correct:
                        pattern_counts['正解→不正解'] += 1
                    elif not initial_correct and challenge_correct:
                        pattern_counts['不正解→正解'] += 1
                    else:
                        pattern_counts['不正解→不正解'] += 1
                
                fig.add_trace(go.Bar(
                    x=list(pattern_counts.keys()),
                    y=list(pattern_counts.values()),
                    text=list(pattern_counts.values()),
                    textposition='auto'
                ), row=2, col=1)
            
            # サブプロット4: 迎合時の正答率詳細
            if results:
                sycophantic_cases = [r for r in results if r.get('is_sycophantic', False)]
                if sycophantic_cases:
                    syc_initial_correct = sum(1 for r in sycophantic_cases if r.get('is_initial_correct', False))
                    syc_challenge_correct = sum(1 for r in sycophantic_cases if r.get('is_challenge_correct', False))
                    syc_total = len(sycophantic_cases)
                    
                    syc_accuracies = [syc_initial_correct/syc_total, syc_challenge_correct/syc_total] if syc_total > 0 else [0, 0]
                    
                    fig.add_trace(go.Bar(
                        x=['迎合時初回', '迎合時チャレンジ'],
                        y=syc_accuracies,
                        text=[f'{acc:.1%}' for acc in syc_accuracies],
                        textposition='auto',
                        marker_color='orange'
                    ), row=2, col=2)
            
            # レイアウト調整
            fig.update_layout(
                title_text=f"LLM迎合性分析結果 - {stats.get('model_name', 'Unknown Model')}",
                height=800,
                showlegend=False
            )
            
            fig.show()
            
            # 3. 具体例の表示（最初の3件）
            if results:
                print(f"\n📝 分析事例（最初の3件）:")
                print("-" * 50)
                
                for i, result in enumerate(results[:3]):
                    print(f"\n事例 {i+1}:")
                    print(f"  質問: {result.get('question', 'N/A')[:60]}...")
                    print(f"  正答: {result.get('correct_answer', 'N/A')}")
                    print(f"  初回回答: {result.get('initial_choice', 'N/A')} ({'正解' if result.get('is_initial_correct') else '不正解'})")
                    print(f"  チャレンジ回答: {result.get('challenge_choice', 'N/A')} ({'正解' if result.get('is_challenge_correct') else '不正解'})")
                    print(f"  迎合判定: {'あり' if result.get('is_sycophantic') else 'なし'}")
            
            # 4. 結果ファイルの保存情報
            print(f"\n💾 結果保存:")
            print(f"   📁 結果ファイルは 'results/' ディレクトリに保存されました")
            print(f"   📊 可視化図表は 'plots/' ディレクトリに保存されました")
            
            print(f"\n" + "=" * 50)
            print("✅ 可視化完了！")
    
    else:
        print("❌ 分析結果が見つかりません")
        print("💡 上記の分析実行セルを先に実行してください")

except Exception as e:
    print(f"❌ 可視化エラー: {e}")
    print("💡 分析が正常に完了しているか確認してください")